In [37]:
import supersnabb as ss
from supersnabb.cashflows.fixed_rate_coupon import FixedRateLeg

schedule = ss.Schedule(
    ss.Date(2018, 1, 1),
    ss.Date(2023, 1, 2),
    ss.Tenor("1Y"),
    ss.Sweden(),
    ss.BusinessDayConvention.MODIFIEDFOLLOWING,
    ss.BusinessDayConvention.MODIFIEDFOLLOWING,
    "forward",
)
discount_curve= ss.DiscountCurve(
    [ss.Date(2018, 1, 1), ss.Date(2019, 1, 1), ss.Date(2020, 1, 1), ss.Date(2021, 1, 1), ss.Date(2022, 1, 1), ss.Date(2024, 1, 1)],
    [1, 0.99, 0.98, 0.97, 0.96, 0.95],
    ss.ACT365,
    ss.Sweden,
    ss.InterpolationType.LINEAR,
)


fixed_leg = FixedRateLeg(
    schedule,
    10e5,
    0.032,
    ss.ACT365,
    ss.BusinessDayConvention.FOLLOWING,
    ss.Sweden,
    discount_curve
)
fixed_leg.cashflows.to_clipboard()

(end_date.serial_number - start_date.serial_number) = 365
(end_date.serial_number - start_date.serial_number) = 365
(end_date.serial_number - start_date.serial_number) = 365
(end_date.serial_number - start_date.serial_number) = 365
(end_date.serial_number - start_date.serial_number) = 365
(end_date.serial_number - start_date.serial_number) = 365
(end_date.serial_number - start_date.serial_number) = 368
(end_date.serial_number - start_date.serial_number) = 368
(end_date.serial_number - start_date.serial_number) = 368
(end_date.serial_number - start_date.serial_number) = 364
(end_date.serial_number - start_date.serial_number) = 364
(end_date.serial_number - start_date.serial_number) = 364
(end_date.serial_number - start_date.serial_number) = 364
(end_date.serial_number - start_date.serial_number) = 364
(end_date.serial_number - start_date.serial_number) = 364


In [41]:
import QuantLib as ql
import pandas as pd
# Begin by setting the valuation date of which the cap and the floor should be priced at
ql.Settings.instance().evaluationDate = ql.Date(1, 1, 2018)
# Then we initialize the curve we want to use for discounting and forecasting
discount_factors = [1, 0.99, 0.98, 0.97, 0.96, 0.95]  # discount factors
dates = [
    ql.Date(1, 1, 2018),
    ql.Date(1, 1, 2019),
    ql.Date(1, 1, 2020),
    ql.Date(1, 1, 2021),
    ql.Date(1, 1, 2022),
    ql.Date(1, 1, 2024),
]  # maturity dates of the discount factors
day_counter = ql.Actual365Fixed()
# Note that we will not strip a curve here, but simply use the discount factors and the dates defined above
# By default QuantLib DiscountCurve will log linearly interpolate between the points.
discount_curve = ql.DiscountCurve(dates, discount_factors, day_counter)
# The curve will note be linked in case we want to update the quotes later on
discount_handle = ql.YieldTermStructureHandle(discount_curve)

start_date = ql.Date(1, 1, 2022)
end_date = start_date + ql.Period(12, ql.Months)

# We define the schedule of the cap and floor
schedule = ql.Schedule(
    start_date,                 # Start date of payments
    end_date,                   # End date of payments
    ql.Period(3, ql.Months),    # frequency of payments
    ql.Sweden(),                # Calendar for adjusting for holidays
    ql.ModifiedFollowing,       # Business convention for adjusting for holidays
    ql.ModifiedFollowing,       # Business convention for adjusting for holidays
    ql.DateGeneration.Forward, # Date generation rule for generating the schedule
    False,                      # End of month rule
)

# Create a custom index to track the payments correctly, specifically fixing days.
custom_discount_index= ql.IborIndex(
    "MyIndex",
    ql.Period("1y"),
    0,
    ql.SEKCurrency(),
    ql.Sweden(),
    ql.Following,
    False,
    ql.Actual365Fixed(),
    discount_handle,
)

engine = ql.DiscountingSwapEngine(discount_handle)
tenor = ql.Period('5y')
fixedRate = 0.032
forwardStart = ql.Period("0D")

swap = ql.MakeVanillaSwap(tenor, custom_discount_index, fixedRate, forwardStart, Nominal=10e6, pricingEngine=engine)
cashflows = pd.DataFrame({
    'nominal': cf.nominal(),
    'accrualStartDate': cf.accrualStartDate().ISO(),
    'accrualEndDate': cf.accrualEndDate().ISO(),
    'rate': cf.rate(),
    'amount': cf.amount(),
    'payment date': cf.date().ISO(),
    'discount factor': discount_curve.discount(cf.date()),
    'accrual': cf.accrualPeriod(),
    'coupon': discount_curve.discount(cf.date()) * cf.amount(), 
    } for cf in map(ql.as_coupon, swap.leg(0)))
cashflows

,nominal,accrualStartDate,accrualEndDate,rate,amount,payment date,discount factor,accrual,coupon
0,10000000.0,2018-01-02,2019-01-02,0.032,320000.000000,2019-01-02,0.989972,1.000000,316791.188420
1,10000000.0,2019-01-02,2020-01-02,0.032,320000.000000,2020-01-02,0.979973,1.000000,313591.212040
2,10000000.0,2020-01-02,2021-01-04,0.032,321777.777778,2021-01-04,0.969917,1.005556,312097.860816
3,10000000.0,2021-01-04,2022-01-03,0.032,319111.111111,2022-01-03,0.959972,0.997222,306337.878169
4,10000000.0,2022-01-03,2023-01-02,0.032,319111.111111,2023-01-02,0.954973,0.997222,304742.562902


In [29]:
discount_curve.dis

<QuantLib.QuantLib.DiscountCurve; proxy of <Swig Object of type 'ext::shared_ptr< InterpolatedDiscountCurve< LogLinear > > *' at 0x000002405BBC7780> >

In [25]:
ql.Actual360().yearFraction(ql.Date(2, 1, 2018), ql.Date(2, 1, 2019))

1.0138888888888888

In [19]:
ql.daysBetween(ql.Date(2, 1, 2018), ql.Date(2, 1, 2019))

365.0